In [6]:
import pandas as pd 
import glob

In [8]:
all_data = []

In [34]:
! ls -lh ../modeling/data/sentence-data-*

-rw-r--r--  1 alex  staff   8.9M Nov  1 23:07 ../modeling/data/sentence-data-ap.csv.gz
-rw-r--r--  1 alex  staff   144M Nov  1 23:07 ../modeling/data/sentence-data-bbc.csv.gz
-rw-r--r--  1 alex  staff   596M Nov  1 23:14 ../modeling/data/sentence-data-guardian.csv
-rw-r--r--  1 alex  staff   139M Nov  1 16:03 ../modeling/data/sentence-data-independent.csv
-rw-r--r--  1 alex  staff    24M Nov  1 23:08 ../modeling/data/sentence-data-nyt.csv.gz
-rw-r--r--  1 alex  staff   100M Nov  1 16:49 ../modeling/data/sentence-data-reuters.csv
-rw-r--r--  1 alex  staff   6.4K Oct 31 23:11 ../modeling/data/sentence-data-small.csv
-rw-r--r--  1 alex  staff    27M Nov  1 03:07 ../modeling/data/sentence-data-wp.csv


In [ ]:
for file in filter(lambda x: 'small' not in x, glob.glob('../modeling/data/sentence-data-*')):
    print(file)
    source = file.split('/')[-1].split('-')[-1].split('.')[0]
    try:
        df = pd.read_csv(file)
        df['source'] = source
        all_data.append(df)
    except:
        print('failed on %s' % file)

In [36]:
all_data_df = pd.concat(all_data)

In [38]:
all_data_df

(51578234, 11)

In [46]:
import unidecode

In [48]:
all_data_df['sentence'] = (
    all_data_df['sentence']
    .fillna('')
    .apply(lambda x: unidecode.unidecode(x.replace('<p>', '').replace('</p>', '')))
)

In [65]:
all_data_df.to_csv('../modeling/data/training_data_full.csv')

In [54]:
doc_lengths = all_data_df.assign(c=1).groupby(['source', 'entry_id', 'version'])['c'].count()

In [61]:
all_data_df = all_data_df.set_index(['source', 'entry_id', 'version'])

In [92]:
empty_sents = all_data_df.loc[lambda df: (df['sentence'] == '') | df['sentence'].isnull()]

In [93]:
docs_with_empty_sents = empty_sents.reset_index()[['source', 'entry_id', 'version']].drop_duplicates()

In [94]:
docs_with_empty_sents = docs_with_empty_sents.set_index(['source', 'entry_id', 'version'])

In [114]:
all_data_df_small = (
    all_data_df
    .loc[doc_lengths.loc[lambda s: (s < 15) & (s > 5)].index]
    .loc[lambda df: ~df.index.isin(docs_with_empty_sents.index)]
)

In [115]:
all_data_df_small.to_csv('../modeling/data/training_data_short_15.csv')

In [116]:
all_data_df_small_downsampled = all_data_df.loc[
    doc_lengths
    .loc[doc_lengths.loc[lambda s: (s < 15) & (s > 5)].index]
    .loc[lambda df: ~df.index.isin(docs_with_empty_sents.index)]
    .sample(50_000).index
]

In [117]:
all_data_df_small_downsampled.to_csv('../modeling/data/training_data_short_15__sampled_50000.csv')

In [118]:
all_data_df_small_downsampled.shape 

(529502, 8)

In [153]:
all_data_df_small_downsampled = all_data_df.loc[
    doc_lengths
    .loc[doc_lengths.loc[lambda s: (s < 15) & (s > 5)].index]
    .loc[lambda df: ~df.index.isin(docs_with_empty_sents.index)]
    .sample(30_000).index
]

In [154]:
all_data_df_small_downsampled.to_csv('../modeling/data/training_data_short_15__sampled_30000.csv')

# Make Document Data

In [108]:
doc = all_data_df_small_downsampled.loc[all_data_df_small_downsampled.index[0]]

/Users/alex/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: PerformanceWarning: indexing past lexsort depth may impact performance.
  """Entry point for launching an IPython kernel.


In [121]:
pd.cut
import numpy as np 

In [128]:
doc['sent_idx']

source  entry_id  version
bbc     418941    0.0         0.0
                  0.0         1.0
                  0.0         2.0
                  0.0         3.0
                  0.0         4.0
                  0.0         5.0
                  0.0         6.0
                  0.0         7.0
                  0.0         8.0
                  0.0         9.0
                  0.0        10.0
                  0.0        11.0
                  0.0        12.0
                  0.0        13.0
Name: sent_idx, dtype: float64

In [149]:
doc['sent_idx'].pipe(lambda s: s.clip(upper=int(len(s)/ 3) * 3-1))

source  entry_id  version
bbc     418941    0.0         0.0
                  0.0         1.0
                  0.0         2.0
                  0.0         3.0
                  0.0         4.0
                  0.0         5.0
                  0.0         6.0
                  0.0         7.0
                  0.0         8.0
                  0.0         9.0
                  0.0        10.0
                  0.0        11.0
                  0.0        11.0
                  0.0        11.0
Name: sent_idx, dtype: float64

In [152]:
(doc.assign(section=lambda df: 
            df['sent_idx']
                 .pipe(lambda s: s.clip(upper=int(len(s)/ 3) * 3-1))
                 .pipe(lambda s: pd.cut(s, np.arange(0, len(s), int(len(s)/3)), right=False))
    )
).groupby('section')['deleted_label	add_above_label	add_below_label	edited_label	unchanged_label	refactored_label']

sent_idx  \
source entry_id version             
bbc    418941   0.0           0.0   
                0.0           1.0   
                0.0           2.0   
                0.0           3.0   
                0.0           4.0   
                0.0           5.0   
                0.0           6.0   
                0.0           7.0   
                0.0           8.0   
                0.0           9.0   
                0.0          10.0   
                0.0          11.0   
                0.0          12.0   
                0.0          13.0   

                                                                  sentence  \
source entry_id version                                                      
bbc    418941   0.0      A mother killed her two sons and then took her...   
                0.0      Susan Talby, 42, and her two sons Joseph, four...   
                0.0      The inquest heard she left a suicide note whic...   
                0.0      He said that if Mrs Talby had survived she wou...   
                0.0      Mrs Talby and her children were found by her h...   
                0.0      'Delusional and homicidal' Mrs Talby had suffe...   
                0.0      The inquest heard the note, addressed to "Rich...   
                0.0           You never hurt me and you put up with a lot.   
                0.0      " A statement read by Peterborough Coroner Gor...   
                0.0      Earlier the inquest heard that Mrs Talby's med...   
                0.0      She was admitted to nearby Edith Cavell Hospit...   
                0.0      However, a note which outlined her condition a...   
                0.0      When she was discharged she was diagnosed with...   
                0.0                                 The inquest continues.   

                         deleted_label  add_above_label  add_below_label  \
source entry_id version                                                    
bbc    418941   0.0                0.0              0.0              0.0   
                0.0                0.0              0.0              0.0   
                0.0                0.0              0.0              0.0   
                0.0                0.0              0.0              0.0   
                0.0                0.0              0.0              0.0   
                0.0                0.0              0.0              0.0   
                0.0                0.0              0.0              0.0   
                0.0                0.0              0.0              0.0   
                0.0                0.0              0.0              0.0   
                0.0                0.0              0.0              0.0   
                0.0                0.0              0.0              0.0   
                0.0                0.0              0.0              0.0   
                0.0                0.0              0.0              0.0   
                0.0                1.0              0.0              0.0   

                         edited_label  unchanged_label  refactored_label  \
source entry_id version                                                    
bbc    418941   0.0               0.0              1.0               0.0   
                0.0               0.0              1.0               0.0   
                0.0               0.0              1.0               0.0   
                0.0               0.0              1.0               0.0   
                0.0               0.0              1.0               0.0   
                0.0               0.0              1.0               0.0   
                0.0               0.0              1.0               0.0   
                0.0               0.0              1.0               0.0   
                0.0               0.0              1.0               0.0   
                0.0               0.0              1.0               0.0   
                0.0               0.0              1.0